## Leading Causes of Death Around the World

In [82]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import requests
import time
from scipy.stats import linregress
from pprint import pprint

In [83]:
# Pull in csv files 
annual_num_deaths_csv = ("../Resources/annual-number-of-deaths-by-cause.csv")
mortality_df = pd.read_csv(annual_num_deaths_csv)
mortality_df.head()

socio_csv = ("../Resources/soci_econ_country_profiles.csv")
socio_df = pd.read_csv(socio_csv)
# socio_df.head()

Deaths - Neoplasms - Sex: Both - Age: All Ages (Number)

In [85]:
mortality_column_list = mortality_df.columns
# mortality_column_list[6].replace("Deaths -","")

new_mort_list = []

for columns in range(len(mortality_column_list)):
    new = mortality_column_list[columns].replace("Deaths - ","")
    new_two = new.replace("- Sex: Both - Age: All Ages (Number)", "")
    new_mort_list.append(new_two) 
# print(new_mort_list)

In [167]:
mortality_df.columns = new_mort_list
mortality_df.head()

,Entity,Code,Year,Number of executions (Amnesty International),Meningitis,Neoplasms,"Fire, heat, and hot substances",Malaria,Drowning,Interpersonal violence,...,Protein-energy malnutrition,Terrorism (deaths),Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease
0,Afghanistan,AFG,2007,15,2933.0,15925.0,481.0,393.0,2127.0,3657.0,...,2439.0,1199.0,53962.0,4490.0,7222.0,3346.0,6458.0,3437.0,1402.0,450.0
1,Afghanistan,AFG,2008,17,2731.0,16148.0,462.0,255.0,1973.0,3785.0,...,2231.0,1092.0,54051.0,4534.0,7143.0,3316.0,6408.0,3005.0,1424.0,455.0
2,Afghanistan,AFG,2009,0,2460.0,16383.0,448.0,239.0,1852.0,3874.0,...,1998.0,1065.0,53964.0,4597.0,7045.0,3291.0,6358.0,2663.0,1449.0,460.0
3,Afghanistan,AFG,2011,2,2327.0,17094.0,448.0,390.0,1775.0,4170.0,...,1805.0,1525.0,54347.0,4785.0,6916.0,3318.0,6370.0,2365.0,1508.0,473.0
4,Afghanistan,AFG,2012,14,2254.0,17522.0,445.0,94.0,1716.0,4245.0,...,1667.0,3521.0,54868.0,4846.0,6878.0,3353.0,6398.0,2264.0,1544.0,482.0


In [169]:
# Filter by years 2015 - 2019
year_df = mortality_df.loc[(
    mortality_df["Year"] == 2015) | 
    (mortality_df["Year"] == 2016) | 
    (mortality_df["Year"] == 2017) | 
    (mortality_df["Year"] == 2018) |
    (mortality_df["Year"] == 2019)]
year_df.head()

,Entity,Code,Year,Number of executions (Amnesty International),Meningitis,Neoplasms,"Fire, heat, and hot substances",Malaria,Drowning,Interpersonal violence,...,Protein-energy malnutrition,Terrorism (deaths),Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease
7,Afghanistan,AFG,2015,1,1821.0,18876.0,463.0,284.0,1730.0,5039.0,...,1469.0,6216.0,56875.0,5153.0,6835.0,3492.0,6555.0,2065.0,1652.0,511.0
8,Afghanistan,AFG,2016,6,1795.0,19459.0,471.0,388.0,1742.0,5033.0,...,1416.0,6142.0,57970.0,5296.0,6894.0,3578.0,6673.0,2044.0,1690.0,524.0
27,Afghanistan,AFG,2017,NaN,1665.0,20111.0,476.0,399.0,1723.0,5191.0,...,1332.0,6092.0,59553.0,5419.0,6982.0,3693.0,6841.0,2046.0,1728.0,537.0
28,Afghanistan,AFG,2018,NaN,1576.0,20673.0,475.0,466.0,1669.0,5500.0,...,1245.0,NaN,60893.0,5531.0,7036.0,3732.0,6861.0,1972.0,1757.0,551.0
29,Afghanistan,AFG,2019,NaN,1563.0,21247.0,485.0,530.0,1687.0,5015.0,...,1210.0,NaN,61995.0,5637.0,7082.0,3797.0,6929.0,1940.0,1775.0,560.0


In [ ]:
most_deaths = ["Bulgaria","Ukraine","Latvia","Lesotho","Lithuania","Serbia","Croatia","Romania","Georgia","Russia"]
least_deaths = ["Qatar","United Arab Emirates","Bahrain","Oman","Kuwait","Maldives","Saudi Arabia","Palestine","Jordan","Solomon Islands"]

In [157]:
most_death_df = year_df.loc[(
    year_df["Entity"] == "Bulgaria") | 
    (year_df["Entity"] == "Ukraine") | 
    (year_df["Entity"] == "Latvia") |
    (year_df["Entity"] == "Lesotho") | 
    (year_df["Entity"] == "Lithuania") | 
    (year_df["Entity"] == "Serbia") | 
    (year_df["Entity"] == "Croatia") | 
    (year_df["Entity"] == "Romania") | 
    (year_df["Entity"] == "Georgia") | 
    (year_df["Entity"] == "Russia")]
most_death_df.reset_index(drop=True, inplace=True)
most_death_df.head()

,Entity,Code,Year,Number of executions (Amnesty International),Meningitis,Neoplasms,"Fire, heat, and hot substances",Malaria,Drowning,Interpersonal violence,...,Protein-energy malnutrition,Terrorism (deaths),Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease
0,Bulgaria,BGR,2015,NaN,43.0,22744.0,125.0,0.0,170.0,177.0,...,3.0,0.0,76745.0,1953.0,2916.0,2715.0,4539.0,10.0,2983.0,939.0
1,Bulgaria,BGR,2016,NaN,42.0,22444.0,121.0,0.0,167.0,182.0,...,4.0,0.0,76814.0,1936.0,2945.0,2724.0,4529.0,10.0,3034.0,945.0
2,Bulgaria,BGR,2017,NaN,41.0,22221.0,119.0,0.0,164.0,181.0,...,4.0,NaN,76978.0,1927.0,2966.0,2723.0,4517.0,9.0,3081.0,955.0
3,Bulgaria,BGR,2018,NaN,40.0,22179.0,118.0,0.0,163.0,182.0,...,4.0,NaN,78759.0,1950.0,3021.0,2722.0,4529.0,9.0,3126.0,987.0
4,Croatia,HRV,2015,NaN,18.0,14693.0,41.0,0.0,87.0,45.0,...,1.0,NaN,24640.0,920.0,1910.0,1201.0,2293.0,1.0,1898.0,484.0


In [171]:
least_death_df = year_df.loc[(
    year_df["Entity"] == "Qatar") | 
    (year_df["Entity"] == "United Arab Emirates") | 
    (year_df["Entity"] == "Bahrain") |
    (year_df["Entity"] == "Oman") | 
    (year_df["Entity"] == "Kuwait") | 
    (year_df["Entity"] == "Maldives") | 
    (year_df["Entity"] == "Saudi Arabia") | 
    (year_df["Entity"] == "Palestine") | 
    (year_df["Entity"] == "Jordan") | 
    (year_df["Entity"] == "Solomon Islands")]
least_death_df.reset_index(drop=True, inplace=True)
least_death_df.head()

,Entity,Code,Year,Number of executions (Amnesty International),Meningitis,Neoplasms,"Fire, heat, and hot substances",Malaria,Drowning,Interpersonal violence,...,Protein-energy malnutrition,Terrorism (deaths),Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease
0,Bahrain,BHR,2015,NaN,3.0,606.0,16.0,0.0,14.0,31.0,...,3.0,5.0,955.0,130.0,123.0,107.0,164.0,3.0,53.0,16.0
1,Bahrain,BHR,2016,NaN,3.0,649.0,15.0,0.0,13.0,30.0,...,3.0,1.0,1021.0,139.0,132.0,114.0,175.0,3.0,58.0,17.0
2,Bahrain,BHR,2017,NaN,3.0,700.0,15.0,0.0,13.0,29.0,...,3.0,6.0,1104.0,151.0,142.0,122.0,187.0,3.0,63.0,19.0
3,Bahrain,BHR,2018,NaN,3.0,753.0,15.0,0.0,13.0,28.0,...,3.0,NaN,1176.0,162.0,150.0,129.0,198.0,4.0,68.0,20.0
4,Bahrain,BHR,2019,NaN,3.0,817.0,15.0,0.0,12.0,27.0,...,3.0,NaN,1271.0,176.0,161.0,138.0,212.0,4.0,74.0,21.0


In [ ]:
del most_death_df["Number of executions (Amnesty International)"]
del most_death_df["Terrorism (deaths)"]
del least_death_df["Number of executions (Amnesty International)"]
del least_death_df["Terrorism (deaths)"]

In [172]:
most_death_clean_df = most_death_df[["Entity","Year","Neoplasms ","Lower respiratory infections ","Neonatal disorders ","Cardiovascular diseases ","Chronic respiratory diseases "]]
most_death_clean_df.head()

,Entity,Year,Neoplasms,Lower respiratory infections,Neonatal disorders,Cardiovascular diseases,Chronic respiratory diseases
0,Bulgaria,2015,22744.0,1919.0,190.0,76745.0,2916.0
1,Bulgaria,2016,22444.0,1938.0,180.0,76814.0,2945.0
2,Bulgaria,2017,22221.0,1951.0,177.0,76978.0,2966.0
3,Bulgaria,2018,22179.0,1983.0,166.0,78759.0,3021.0
4,Croatia,2015,14693.0,513.0,81.0,24640.0,1910.0


In [173]:
least_death_clean_df = least_death_df[["Entity","Year","Neoplasms ","Lower respiratory infections ","Neonatal disorders ","Cardiovascular diseases ","Chronic respiratory diseases "]]
least_death_clean_df.head()

,Entity,Year,Neoplasms,Lower respiratory infections,Neonatal disorders,Cardiovascular diseases,Chronic respiratory diseases
0,Bahrain,2015,606.0,68.0,50.0,955.0,123.0
1,Bahrain,2016,649.0,71.0,45.0,1021.0,132.0
2,Bahrain,2017,700.0,76.0,41.0,1104.0,142.0
3,Bahrain,2018,753.0,79.0,37.0,1176.0,150.0
4,Bahrain,2019,817.0,83.0,33.0,1271.0,161.0
